# This is a work-in-progress notebook

We wish to know this:

1. How well does the model identify the correct number of senses for the target word?
2. **How well does the model identify the correct senses for the target word?**
3. **How well does the model assign the right words to a given sense of the target word?**
4. How well does the model assign the senses to the time intervals for the target word?

The script will evaluate **Q2** and **Q3**. Q4 will follow.

In [62]:
# Basic variables and imports:

import codecs, csv, os, time, re, io
from os import listdir
from os.path import isfile, join

# directories
dir_in = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "evaluation", "evaluation_input"))
dir_out = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "evaluation", "evaluation_output"))

s_senses = io.open(dir_in+"/senses_69419.txt","r")
k_senses = io.open(dir_in+"/mus.dat","r")

file_senses = s_senses.readlines() 
output_senses = k_senses.readlines()

## TODO

- ~~create the notebook~~
- ~~organise the notebook~~
- ~~write "general idea" pseudocode for the evaluation~~
- ~~get input files~~
- figure out data structures to store the variables
- write actual code

## Q2: How well does the model identify the correct senses for the target word?

### Pseudocode 

In [ ]:
# For each target word, we have a list of senses  s (given by the expert)
# For each target word, we have a list of senses k (given by the model)
# This Q consists in matching s and k, and doing so in a confident way --> confidence score

for each k:
    for each s:
        create conf(k,s)

# What is conf(k,s)?
        conf(k,s) = (p1*match(w1,s)+p2*match(w1,s)+px(wx,s))/10 WHERE
    
            px = probability of word wx 
                
                and
            
            match(wx,s) =   1/number_of_senses_assigned_to_wx if s_is_one_of_them 
            
                    or 
                            0 if w_is_not_associated_to_s
                
# Once we have gone through all s for one k, we have to choose the best k for s. How? (TBD, cfr Valerio and Barbara)

# Once all ks have been assigned to all ss (or NA), we can calculate a general confidence score for the model.
# One easy way to do that: 

conf_score_model = number_of_non_NA/k

### Real code

Steps:

- extract all senses from the file
- use those senses as keys for a dictionary, `dict_of_words`
- fill the dictionary: for each key, we store a list of words pertaining to that sense
- transform the lists as sets so as to remove duplicates within the same sense
- create a dictionary with a word as a key and its weight as a value, depending on how many senses it appears

Todo:

- do not take into account the first line
- parse the model output and get the probability weights for each word

In [102]:
expert_senses = list() # list where we store all sense ids provided by expert
for s in file_senses[:20]: # 20 for testing purposes
    s = s.split("\t")
    sense = s[11] # The sense ID is after the 10th tab
    expert_senses.append(sense)
    
#print(len(expert_senses),expert_senses,len(set(expert_senses)))

expert_senses = list(set(expert_senses)) # we only keep the unique senses
number_of_s = len(expert_senses)  # we create a variable that stores the number of unique senses
print("Number of senses:",number_of_s)

# This dictionary has a sense as a key, and a list of words as a value. 
dict_of_words = dict()
# This list stores all words
list_of_all_words = list()
# This dictionary stores all words as keys and their weight as value
word_weight = dict()

for i in range(0,number_of_s): # for each sense, we create a dictionary entry which has a list as value
    dict_of_words[expert_senses[i]] = list()

    for s in file_senses[:20]: # we go back in the file
        s = s.split("\t") # splitting on tabs
        
        sentence_of_ids = s[9] 
        list_of_ids = sentence_of_ids.split(" ")  # splitting on spaces
        for word_id in list_of_ids:
            if s[11] == expert_senses[i]:      # we store all words for one sense 
                dict_of_words[expert_senses[i]].append(word_id)
            list_of_all_words.append(word_id) # we store all words, we'll iterate over that for scores
        

    # Here, we remove duplicates
    dict_of_words[expert_senses[i]] = set(dict_of_words[expert_senses[i]]) 
      
    print(expert_senses[i],set(dict_of_words[expert_senses[i]]))
    print("\n\n")


Number of senses: 3
mus-1 {'ἀπικομένοισι', 'τάδε', 'ἔοικε', 'ἔχων', 'γυμνῶν', 'ὥστε', 'Γοβρύεω', 'οὐρανόν', 'πάσσακι', 'καταδύητε', 'ἐκεῖνον', 'γινομένου', 'κοπρολόγῳ', 'σφέων', 'βασιλέες', 'πολλούς', 'τῇ', 'βάτραχος', 'γενόμενοι', 'ἐπὶ', 'ὅκκ’', 'ὀπίσω', 'ἀπονοστήσετε', 'γε', 'μέλλεις', 'κατελόντων', 'ὑμὲς', 'ἀρωραῖοι', 'σφέας', 'χειρὸς', 'πεσεῖν', 'τοῦτο', 'ὄρνιθες', 'βαλλόμενοι', 'πρὸς', 'ὡς', 'ἑωυτῶν', 'γίνεται', 'φευγόντων', 'ἢ', 'Μάγον', 'μὲν', 'βασιλεὺς', 'μέν', 'τῶν', 'εἴχετο', 'ἀσπίδων', 'ὑστεραίῃ', 'ποθεν', 'τῷ', 'ἐσπηδήσητε', 'ἐναντίοισι', 'δὲ', 'γῆς', 'αὐτόθι', 'τοίνυν', 'τοὺς', 'ἑνὸς', 'ἐσβάλητε', 'μύες', 'γῇ', 'γνώμη', 'ποτ’', 'ἀρουραίους', 'αὐτοὺς', 'βάτραχοι', 'ταῦτα', 'νυν', 'ἕστηκε', 'ἴσως', 'νυκτὸς', 'κἀπαίδευτε', 'μάλιστα', 'ἀνθρώπῳ', 'φέροντα', 'εἰκάζων', 'Σκύθας', 'φαρετρεῶνας', 'τοῖσι', 'ἀεί', 'ἔστι', 'γραμμάτων', 'διὰ', 'τοξευμάτων', 'λοιδορούμενος', 'νῦν', 'παραδιδοῦσι', 'Δαρείῳ', 'τὴν', 'δῶρα', 'σκαιὲ', 'ἔφη', 'ἀναπτῆσθε', 'ἐπιχυθέντας', 'ἐς', 'Ἡφαίστου', 'οὕτ

In [120]:
# For every word in the list of words that we have
# we count the number of senses it appears in
# we use that number to divide its importance: 1 sense = 1 importance; 2 senses = 0.5 importance
# this can be finetuned

for word in list_of_all_words:
    x = 0
    for i in range(0,number_of_s):
        if word in dict_of_words[expert_senses[i]]:
            x += 1 
        if x != 0:
            word_weight[word] = 1/x
        
    print(word,word_weight[word]) 

ἀπικομένοισι 1.0
τάδε 1.0
ὑστέρῃσιν 1.0
ἔοικε 1.0
ὄπισθεν 1.0
ἔχων 1.0
βραχίονος 1.0
ἐπὴν 1.0
γυμνῶν 1.0
ὥστε 1.0
χονδρώδεα 1.0
Γοβρύεω 1.0
οὐρανόν 1.0
ἔξω 1.0
πάσσακι 1.0
καταδύητε 1.0
ἐκεῖνον 1.0
ὅμοια 1.0
γινομένου 1.0
κοπρολόγῳ 1.0
ἀνάληψιν 1.0
σφέων 1.0
βασιλέες 1.0
πολλούς 1.0
τῇ 0.5
γὰρ 1.0
βάτραχος 1.0
γενόμενοι 1.0
ἐπὶ 1.0
ὀστέα 1.0
ὅκκ’ 1.0
ἀπὸ 1.0
ὀπίσω 1.0
ἀπονοστήσετε 1.0
γε 1.0
μέλλεις 1.0
Ὁκόσοισι 1.0
κατελόντων 1.0
τόπους 1.0
ὑμὲς 1.0
ἄλλο 1.0
ἀρωραῖοι 1.0
ἐπίφυσιν 1.0
σφέας 1.0
ᾗ 1.0
χειρὸς 1.0
πεσεῖν 1.0
τοῦτο 0.5
ὄρνιθες 1.0
βαλλόμενοι 1.0
πρὸς 1.0
ὡς 1.0
ἐπιδεῖν 1.0
ἑωυτῶν 1.0
πληρεῦντες 1.0
γίνεται 1.0
φευγόντων 1.0
ἢ 0.5
Μάγον 1.0
συμβαίνει 1.0
μὲν 0.5
βασιλεὺς 1.0
ἀπόσφιγξις 1.0
μέν 0.5
τῶν 1.0
διατάσιος 1.0
εἴχετο 1.0
ἀσπίδων 1.0
ὑστεραίῃ 1.0
ὀξείην 1.0
παραπεφύκασιν 1.0
ἐντέροισιν 1.0
κατατεταμένος 1.0
ποθεν 1.0
τῷ 0.5
ἐσπηδήσητε 1.0
ἔκφυσιν 1.0
σχήσει 1.0
ἐναντίοισι 1.0
δὲ 0.5
γῆς 1.0
αὐτόθι 1.0
τοίνυν 1.0
τὸ 1.0
προσδέχου 1.0
τοὺς 0.5
ἀπόστασις 1.0
γίνονται 1

## Q3: How well does the model assign the right words to a given sense of the target word?

In [ ]:
# For each k, we use the words given by the expert as unquestionable truth.
# Judging the model's assignment of words to a given sense becomes a question of precision and recall.

# precision is all correct w weighted by their respective probabilities / all w weighted by their probabilities
for each k:
    for each w:
        if w in expert_list:
            w_weight = p*1
            numerator += w_weight
        w_weight = p*1
        denominator += w_weight
    precision = numerator/denominator
    
# recall is all correct w weighted by their respective probabilities / all w assigned to the sense by the expert
for each k:
    for each w:
        if w in expert_list:
            w_weight = p*1
            numerator += w_weight
    denominator = len(expert_list)
    recall = numerator/denominator
    
# f-score can be used as well

for each k:
    f_score = 2 * precision * recall / (precision+recall)